<a href="https://colab.research.google.com/github/bgalerne/M1MAS_Stat_Images/blob/master/Multiclass_Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # Multiclass logistic regression
 Goal:
 1. Define functions for training a multiclass logistic regression 
 1. Train the classifier using gradient descent
 1. Visualize a multi-class logisitc regression for 2D data
 1. (TODO) Implement averaged stochastic gradient descent 

 **Reference:**
 Section "4.3.4 Multiclass logistic regression"
 of 
 
 C. M. Bishop *Pattern Recognition and Machine Learning*,
Information Science and Statistics, Springer, 2006

Freely available:
https://www.microsoft.com/en-us/research/people/cmbishop/prml-book/


In [0]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.datasets import make_moons, make_circles, make_classification, make_blobs, make_gaussian_quantiles
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [0]:
# create some toy datasets :

#
n_class = 3
# Three examples of synthetic 2D datasets:
X, t = make_blobs(n_features=2, centers = n_class)
#X, t = make_classification(n_features=2, n_redundant=0, n_informative=2, random_state=24, n_classes=n_class, n_clusters_per_class=1)
#X, t = make_gaussian_quantiles(n_features=2, n_classes=n_class)

X = StandardScaler().fit_transform(X)
print(X.shape)

#X, y = make_gaussian_quantiles(n_features=2, n_classes=3)


X_train, X_test, t_train, t_test = train_test_split(X, t, test_size=.4, random_state=12)

figure = plt.figure(figsize=(10, 10))
plt.scatter(X_train[:, 0], X_train[:, 1], marker='o', c=t_train, s=50, edgecolor='k')
plt.scatter(X_test[:, 0], X_test[:, 1], marker='P', c=t_test, s=50, edgecolor='k')

N_train = X_train.shape[0]
N_test = X_test.shape[0]

In [0]:
# Apply feature transform:

# Add the 1 coordinate for bias:
feature_transform = lambda X : np.hstack( (X, np.ones((X.shape[0],1))))
# Add the 1 coordinate and square of coordinates:
#feature_transform = lambda X : np.hstack( (X, X**2, np.ones((X.shape[0],1))))

Phi_train = feature_transform(X_train)
n_feat = Phi_train.shape[1]

Phi_test = feature_transform(X_test)


In [0]:
# Function for Multiclass logistic regression:
# - W is the matrix of size n_feat x n_class

def soft_max(W, Phi):
  # evaluate the softmax vector for a list of feature points phi (given in line) 
  a = W.transpose() @ Phi.transpose() 
  y = np.exp(a)
  s = np.sum(y,axis=0)
  y = (y/s).transpose()
  return(y)

def predicted_class(W,Phi):
  y = soft_max(W, Phi)
  if Phi.ndim==1:
    pred = np.argmax(y)
  else:
    pred = np.argmax(y,axis=1)
  return(pred)

def mloglikelihood(W, Phi, t):
  y = soft_max(W, Phi)
  # extract values of softmax for the class k=t
  if t.ndim == 0:
    y = y[t]
    L = np.log(y)
  else:
    y = y[np.arange(Phi.shape[0]),t]
    L = - np.sum(np.log(y))
  return(L)

def gradmloglikelihood(W, Phi, t):
  y = soft_max(W, Phi)
  # extract values of softmax for the class k=t
  if t.ndim == 0:
    y[t] -= 1
    y.shape = (n_class,1)
    Phi.shape = (1,n_feat)
    g = y @ Phi
  else:
    y[np.arange(Phi.shape[0]),t] = y[np.arange(Phi.shape[0]),t] - 1
    #print(y.shape)
    #print(Phi.shape)
    g = y.transpose() @ Phi
  return(g.transpose())



In [0]:
# random initialization:
W = np.random.random((n_feat,n_class))


lr = 0.01

Nit = 10**4
for n in range(Nit):
  W -= lr*gradmloglikelihood(W, Phi_train, t_train)
  if n%1000==0:
    print(mloglikelihood(W, Phi_train, t_train))




In [0]:
pred = predicted_class(W,Phi_test)
print(pred[:30])
print(t_test[:30])
# TODO: Compute accuracy

In [0]:
#visualize results:

x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
h = 0.02
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                        np.arange(y_min, y_max, h))
X_grid = np.hstack((xx.ravel(), yy.ravel()))
print(X_grid.shape)
N_grid = xx.ravel().shape[0]
X_grid = np.c_[xx.ravel(), yy.ravel()]
Phi_grid = feature_transform(X_grid)
print(Phi_grid.shape)
Z = predicted_class(W,Phi_grid)
Z = Z.reshape(xx.shape)

figure = plt.figure(figsize=(16, 8))
ax = plt.subplot(1,2,1)
ax.set_title("Input data")
ax.scatter(X_train[:, 0], X_train[:, 1], marker='o', c=t_train, s=50, edgecolor='k')
ax.scatter(X_test[:, 0], X_test[:, 1], marker='P', c=t_test, s=50, edgecolor='k')
ax = plt.subplot(1,2,2)
cmap = ListedColormap(['b','y','r'])
plt.contourf(xx,yy,Z,  cmap = cmap, alpha=.8)
ax.scatter(X_train[:, 0], X_train[:, 1], marker='o', c=t_train, s=50, edgecolor='k')
ax.scatter(X_test[:, 0], X_test[:, 1], marker='P', c=t_test, s=50, edgecolor='k')


Implement an average stochastic gradient descent for solving the optimization problem:
At each step sort one example of the dataset and use it for a noisy gradient estimate.
Compute the new weight $W^{(n)}$ and the average weights $\bar{W}^{(n)}$ defined by
$$
\bar{W}^{(n)} = \frac{1}{n+1} \sum_{k=0}^{n} W^{(k)} = \frac{n}{n+1}\bar{W}^{(n-1)} + \frac{1}{n+1}W^{(n)}.
$$



In [0]:
# Cell for tests
print(W)
print('Tests of soft_max')
print(soft_max(W, Phi_train[0,:]))
print(soft_max(W, Phi_train[1,:]))
print(soft_max(W, Phi_train[0:2,:]))
print('\nTests of mloglikelihood')
print(mloglikelihood(W, Phi_train[0,:], t_train[0]))
print(mloglikelihood(W, Phi_train[0:2,:], t_train[0:2]))
print(mloglikelihood(W, Phi_train, t_train))
print('\nTests of gradmloglikelihood')
print(gradmloglikelihood(W, Phi_train, t_train))
print(gradmloglikelihood(W, Phi_train[0,:], t_train[0]))
